In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd

from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

In [6]:
food_data_path = r'https://raw.githubusercontent.com/acs14007/GEOG5518restaurantfinder/main/Food_full.csv'

In [96]:
food_data = pd.read_csv(food_data_path)
food_data = gpd.GeoDataFrame(food_data, geometry=gpd.points_from_xy(food_data.longitudes, food_data.latitudes))


food_data['price_labels'] = ''
price_labels = food_data.price_labels.copy()
price_labels[food_data.price == '$'] = 'Under $10'
price_labels[food_data.price == '$$'] = '$11-30'
price_labels[food_data.price == '$$$'] = 'Over $31'
price_labels[food_data.price.isna()] = 'Other'
food_data['price_labels'] = price_labels

In [83]:
token = open('.mapbox_token').read()

In [97]:
food_data.full_address

,index,name,rating,category,price,address,city,zipcode,country,full_address,image_url,phone,latitudes,longitudes,geometry,price_labels
0,1,1000 Degrees Pizza,3.5,Pizza,NaN,2941 Main St,Glastonbury,6033,US,"1000 Degrees Pizza, 2941 Main St, Glastonbury,...",https://s3-media2.fl.yelpcdn.com/bphoto/WJo0np...,(860) 500-1000,41.723593,-72.615392,POINT (-72.61539 41.72359),Other
1,2,451 Restaurant & Lounge,3.5,Latin American,$$,341 Franklin Ave,Hartford,6114,US,"451 Restaurant & Lounge, 341 Franklin Ave, Har...",https://s3-media3.fl.yelpcdn.com/bphoto/hUCIrB...,(860) 206-1556,41.740665,-72.676309,POINT (-72.67631 41.74067),$11-30
2,3,5 And Dime Canteen,4.5,Breakfast & Brunch,$,1100 Main St,Newington,6111,US,"5 And Dime Canteen, 1100 Main St, Newington, C...",https://s3-media3.fl.yelpcdn.com/bphoto/cd1vI6...,(860) 665-7848,41.696770,-72.722822,POINT (-72.72282 41.69677),Under $10
3,4,50 Elm,4.0,Cocktail Bars,$$,50 Elm St,Hartford,6106,US,"50 Elm, 50 Elm St, Hartford, CT 06106,",https://s3-media2.fl.yelpcdn.com/bphoto/jLEA1o...,(860) 206-3322,41.762305,-72.675474,POINT (-72.67547 41.76230),$11-30
4,5,99 Restaurants,3.0,Pubs,$$,3025 Main St,Glastonbury,6033,US,"99 Restaurants, 3025 Main St, Glastonbury, CT ...",https://s3-media1.fl.yelpcdn.com/bphoto/0O7tsB...,(860) 652-9699,41.726554,-72.617603,POINT (-72.61760 41.72655),$11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,196,Harry's Bishops Corner,4.5,Pizza,$$,732 N Main St,West Hartford,6117,US,"Harry's Bishops Corner, 732 N Main St, West Ha...",https://s3-media1.fl.yelpcdn.com/bphoto/gCnIyh...,(860) 236-0400,41.786503,-72.746429,POINT (-72.74643 41.78650),$11-30
195,197,Harry's Pizza,4.0,Pizza,$$,39 S Main St,West Hartford,6107,US,"Harry's Pizza, 39 S Main St, West Hartford, CT...",https://s3-media1.fl.yelpcdn.com/bphoto/akAhN1...,(860) 231-7166,41.759100,-72.743011,POINT (-72.74301 41.75910),$11-30
196,198,Hartford # 1 Chinese Restaurant,3.5,Chinese,$,3155 Main St,Hartford,6120,US,"Hartford # 1 Chinese Restaurant, 3155 Main St,...",https://s3-media3.fl.yelpcdn.com/bphoto/LGNBFm...,(860) 548-0110,41.794689,-72.664787,POINT (-72.66479 41.79469),Under $10
197,199,Hartford Baking,4.0,Cafes,$,965 Farmington Ave,West Hartford,6107,US,"Hartford Baking, 965 Farmington Ave, West Hart...",https://s3-media4.fl.yelpcdn.com/bphoto/wcllX8...,(860) 570-1579,41.756262,-72.760918,POINT (-72.76092 41.75626),Under $10


In [107]:
fig = px.scatter_mapbox(food_data,
                        lat='latitudes',
                        lon='longitudes',
                        color='price_labels',
                        zoom=10,
                        hover_name='full_address',
                        hover_data=['image_url'],
                        category_orders={'price_labels': ['Under $10', '$11-30', 'Over $31', 'Other']},
                        color_discrete_map= {'Under $10': 'yellow', '$11-30': 'orange', 'Over $31': 'red', 'Other': 'grey'},
                        size='rating',
                        size_max=10)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)

fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor="right", x=1))
fig.update_layout(legend_title_text='Price')